<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import tensorflow.compat.v1 as tf
import re

In [2]:
def change_name(checkpoint_path, output_prefix):
    """
    ckpt = tf.train.Checkpoint(vars={name: variable})
    ckpt.restore(converted_ckpt_path)
    Args:
    checkpoint_path: Path to the TF1 checkpoint.
    output_prefix: Path prefix to the converted checkpoint.
    
    Returns:
    Path to the converted checkpoint.
    """
    vars = {}
    reader = tf.train.load_checkpoint(checkpoint_path)
    dtypes = reader.get_variable_to_dtype_map()

    for key in dtypes.keys():
        new_key = key
        if key=='bert/embeddings/layer_normalization/beta' or key=='bert/embeddings/layer_normalization/gamma':
            new_key=key.replace('layer_normalization','LayerNorm')
        elif re.search('layer_normalization_+\d+',key):
            new_key = re.sub('layer_normalization_+\d+','LayerNorm',key)
        elif re.search('layer_normalization',key):
            new_key = re.sub('layer_normalization','LayerNorm',key)
        print(new_key)
        vars[new_key] = tf.Variable(reader.get_tensor(key))

    return tf.train.Saver(var_list=vars).save(sess=None, save_path=output_prefix)